In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Step 1: Install Required Libraries
!pip install -U langchain langchain-community chromadb pymilvus weaviate-client sentence-transformers pypdf

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, Milvus, Weaviate
import weaviate
from pymilvus import connections
import os

In [3]:
# Step 3: Load the PDF Document
loader = PyPDFLoader("/content/drive/MyDrive/BootCamp@PureLogics/week9-day2/Crime-and-Punishment.pdf")
documents = loader.load()


In [4]:
# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")

Total chunks: 522


In [5]:
# Step 4: Create Embeddings from Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-5-396909522.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

In [10]:
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding_model, persist_directory="chroma_db")
vectordb.persist()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [11]:
# Step 6: Perform a Query
def ask(query):
    results = vectordb.similarity_search(query, k=5)
    for i, res in enumerate(results):
        print(f"\nResult {i+1}:")
        print(res.page_content[:500])

In [12]:
# Sample Query
demo_query = "What is Raskolnikov's motive for the murder?"
ask(demo_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Result 1:
crime	could	only	have	been	committed	through	temporary	mental	derangement,
through	homicidal	mania,	without	object	or	the	pursuit	of	gain.	This	fell	in	with
the	most	recent	fashionable	theory	of	temporary	insanity,	so	often	applied	in	our
days	in	criminal	cases.	Moreover	Raskolnikov’s	hypochondriacal	condition	was
proved	 by	 many	 witnesses,	 by	 Dr.	 Zossimov,	 his	 former	 fellow	 students,	 his
landlady	 and	 her	 servant.	 All	 this	 pointed	 strongly	 to	 the	 conclusion	 that
Raskolnikov	

Result 2:
crime	could	only	have	been	committed	through	temporary	mental	derangement,
through	homicidal	mania,	without	object	or	the	pursuit	of	gain.	This	fell	in	with
the	most	recent	fashionable	theory	of	temporary	insanity,	so	often	applied	in	our
days	in	criminal	cases.	Moreover	Raskolnikov’s	hypochondriacal	condition	was
proved	 by	 many	 witnesses,	 by	 Dr.	 Zossimov,	 his	 former	 fellow	 students,	 his
landlady	 and	 her	 servant.	 All	 this	 pointed	 strongly	 to	 the	 conclu